In [59]:
import re
from glob import glob
from os import makedirs, path
import shutil
from datetime import datetime, timedelta

In [60]:
rootdir = "datasets"
dsdir = "processed_binary"
outputdir = "processed_binary_split"

In [61]:
nameRegexp = re.compile(r"^Camera(\d+)_(\d+)-(\d+)-(\d+)")

In [65]:
class Range:
    def __init__(self, date: datetime, tolerance: timedelta, camera: str) -> None:
        self.start = date - tolerance
        self.end = date + tolerance
        self.tolerance = tolerance
        self.camera = camera

    def __contains__(self, date: datetime) -> bool:
        return (self.start - self.tolerance) <= date and date <= (self.end + self.tolerance)

    def extend(self, d: datetime):
        if self.start <= d and d <= self.end:
            return

        self.start -= self.tolerance
        self.end +=  self.tolerance

    def __repr__(self):
        return f"{str(self.start)} - {str(self.end)} (+/- {str(self.tolerance)}) (cam {self.camera})"

In [ ]:
grouped = {}

tolerance = timedelta(minutes=5)

for imagePath in glob(f"{rootdir}/{dsdir}/*/*.jpg"):
    fileName = path.basename(imagePath)

    matches = nameRegexp.findall(fileName)

    if len(matches) == 1:
        camera = matches[0][0]
        hour, minute, second = [int(m) for m in matches[0][1:]]

        d = datetime(year=2025, month=12, day=31, hour=hour, minute=minute, second=second)

        found = False
        for r in grouped.keys():
            if r is not None and d in r and r.camera == camera:
                r.extend(d)

                found = True
                grouped[r].append(imagePath)
                break

        if not found:
            grouped[Range(d, tolerance, camera)] = [imagePath]
    else:
        if grouped.get(None) is None:
            grouped[None] = [imagePath]
        else:
            grouped[None].append(imagePath)


In [73]:
grouped

{None: ['datasets/processed_binary\\close\\152124resized_frame_28_bmp.rf.bc538b942326ed0ce123197b70372872.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_29_bmp.rf.60c00ef5f5c1cd3edefee77c8a2c7ede.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_30_bmp.rf.256c8157aa691aa175edd5bb37ea0313.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_31_bmp.rf.6a728e3808f2104fc2d7c1375154976f.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_32_bmp.rf.9a9b48cefbac5ec89508ad42fc7d0f89.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_33_bmp.rf.5b48b08ab9784f64e93462d3292fa45f.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_34_bmp.rf.f40092028629870ff5dc0d93021de7c7.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_35_bmp.rf.bc289522b6156a0a64938bb91258f75f.jpg',
  'datasets/processed_binary\\close\\152124resized_frame_36_bmp.rf.c74b8df16ecc9a091f66adce96a97256.jpg',
  'datasets/processed_binary\\close\\152

In [77]:
for i, (key, val) in enumerate(grouped.items()):
    if key is not None:
        for filePath in val:
            dest = filePath.replace(dsdir, outputdir)
            dest_filename = path.basename(dest)
            dest_path = path.dirname(dest)
            dest_path = f"{dest_path}/Group_{i+1}"

            makedirs(dest_path, exist_ok=True)
            shutil.copyfile(filePath, f"{dest_path}/{dest_filename}")
    else:
        for filePath in val:
            dest = filePath.replace(dsdir, outputdir)
            makedirs(path.dirname(dest), exist_ok=True)
            shutil.copyfile(filePath, dest)